## Linear Regression - Ordinary Least Squares (OLS) Method 

We will perform linear regression using
- Scikit Learn's OLS model
- Manually coded OLS method


The sklearn OLS implementation code is given in this notebook. You will have to implement the OLS method manually on the given dataset (OLS_Data.csv).


### OLS

OLS is a type of linear least squares method for estimating the unknown parameters in a linear regression model. OLS chooses the parameters of a linear function of a set of explanatory variables by the principle of least squares: minimizing the sum of the squares of the differences between the observed dependent variable (values of the variable being predicted) in the given dataset and those predicted by the linear function.

OLS finds the optimal parameters by computing a closed-form solution for the **Normal equation**.

URL: https://scikit-learn.org/stable/modules/linear_model.html#linear-model


### Dataset

We will use a dataset (OLS_Data.csv) containing 14 variables (14 dimensional feature)

Input variables:
X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14

Output variable: 
y

### Note:
This dataset might have colinearity in the input variables resulting into the singularity problem. It might cause the OLS method not working. You may need to fix the singularity problem.

# Part 1: OLS Linear Regression Using Python 

In [0]:
import numpy as np
from numpy.linalg import inv
from numpy.linalg import det
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

## Load Data

First load the data and explore the feature names, target names, etc.

Download the "OLS_Data.csv" file to load data from it.

In [2]:
# load the csv file as a Pandas DataFrame object denoted as "df"

from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
df= pd.read_csv('/content/drive/My Drive/Colab Notebooks/OLS_Data.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Quick Check of the Data

Let’s take a look at the top five rows using the DataFrame’s head() method.


In [3]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,y
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,0.00632,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,0.02731,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,0.02729,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,0.03237,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,0.06905,36.2


# Description of the Data

DataFrame’s info() method is useful to get a quick description of the data, in particular the total number of rows, and each attribute’s type and number of non-null values.


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 15 columns):
X1     506 non-null float64
X2     506 non-null float64
X3     506 non-null float64
X4     506 non-null int64
X5     506 non-null float64
X6     506 non-null float64
X7     506 non-null float64
X8     506 non-null float64
X9     506 non-null int64
X10    506 non-null int64
X11    506 non-null float64
X12    506 non-null float64
X13    506 non-null float64
X14    506 non-null float64
y      506 non-null float64
dtypes: float64(12), int64(3)
memory usage: 59.4 KB


## Data Matrix: Feature Correlations

Check if the data matrix has colinearity (1 or close to 1) in its features.

In [5]:
df.corr() 

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,y
X1,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621,1.000000,-0.388305
X2,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995,-0.200469,0.360445
X3,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800,0.406583,-0.483725
X4,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929,-0.055892,0.175260
X5,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879,0.420972,-0.427321
X6,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808,-0.219247,0.695360
X7,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339,0.352734,-0.376955
X8,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996,-0.379670,0.249929
X9,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676,0.625505,-0.381626
X10,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993,0.582764,-0.468536


Features X1 and X14 has high correlation 1 between each other.

# Create a Separate Feature Set (Data Matrix X) and Target (1D Vector y)

Create a data matrix (X) that contains all features and a 1D target vector (y) containing the target.



In [6]:
# data matrix X
X = np.array(df.iloc[:,0:14])

# target vector y
y = np.array(df.iloc[:,14:])

print(X.shape)
print(y.shape)

(506, 14)
(506, 1)


# Scale The Features

We should ensure that all features have a similar scale. Otherwise optimization algorithms (e.g., Gradient Descent based algorithms) will take much longer time to converge.

Also, regularization techniques are sensitive to the scale of data. Thus, we must scale the features before applying regularization.

Use sklearns StandardScaler().

In [0]:
scaler = StandardScaler().fit(X)
X = scaler.transform(X)



# Create Train and Test Dataset

Create train and test data (80% & 20%) by usinf sklearn's train_test_split function

It should return the following 4 matrices.
X_train
y_train
X_test
y_test

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

## Linear Regression Models

We will use the following linear regression models.

- Ordinary least squares (OLS) Linear Regression (by solving the Normal Equation)



## Evaluation Metrics

We will use two evaluation metrics.

- Mean Squared Error (MSE)
- Coefficient of Determination ($R^2$ or $r^2$)


### Note on $R^2$:
R-squared is a statistical measure of how close the data are to the fitted regression line. 

R-squared measures the proportion of the variance in the dependent variable that is predictable from the independent variable(s).

$R^2 = \frac{Explained Variation}{Total Variation}$

R-squared is always between 0 and 100%:

- 0% indicates that the model explains none of the variability of the response data around its mean.
- 100% indicates that the model explains all the variability of the response data around its mean.


#### <font color=red>In general, the higher the R-squared, the better the model fits your data.</font>


#### Compute $R^2$ using the sklearn:

- The "r2_score" function from sklearn.metrics

#### Compute MSE using the sklearn:

- The "mean_squared_error" function from sklearn.metrics


## Sklearn Ordinary Least Squares (OLS) Linear Regression (by solving the Normal Equation)


#### Sklearn's OLS model implementation code is given for you to review.

Then, you will have to manually code the OLS method.


#### <font color=red>The MSE and $r^2$ error values from your manually coded OLS method must match with sklearn LinearRegressor's obtained values.</font>

In [9]:
# Create the sklearn OLS linear regression object
lin_reg = LinearRegression()


# Train the model
lin_reg.fit(X_train, y_train)


# The intercept
print("Intercept: \n", lin_reg.intercept_)

# The coefficients
print("Coefficients: \n", lin_reg.coef_)


print("\n----------------------------- Model Evaluation -----------------------------")


# Make prediction 
y_train_predicted = lin_reg.predict(X_train)


print("\nMean squared error: %.2f"
      % mean_squared_error(y_train, y_train_predicted))


# To compute 

# Explained variance score: 1 is perfect prediction
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % r2_score(y_train, y_train_predicted))

# Explained variance score: 1 is perfect prediction
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % lin_reg.score(X_train, y_train))

('Intercept: \n', array([22.48526824]))
('Coefficients: \n', array([[-0.48574711,  0.70155562,  0.27675212,  0.70653152, -1.99143043,
         3.11571836, -0.17706021, -3.04577065,  2.28278471, -1.79260468,
        -1.97995351,  1.12649864, -3.62814937, -0.48574711]]))

----------------------------- Model Evaluation -----------------------------

Mean squared error: 21.64
Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.75
Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.75


## Evaluate the Sklearn OLS Model Using Test Data 

We evaluate the trained model on the test data.

The goal is to see how the model performs on the test data.

In [10]:
# Make prediction 
y_test_predicted = lin_reg.predict(X_test)


print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_test_predicted))


# Explained variance score: 1 is perfect prediction
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % r2_score(y_test, y_test_predicted))

Mean squared error: 24.29
Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.67


## Manually Coded OLS Solution



In [11]:
# Manually code the OLS Method for Linear Regression


# Add a bias term with the feature vectors to create a new data matrix "X_train_bias"
X_train_bias=np.c_[np.ones((X_train.shape[0],1)),X_train]


# Print the determinant of the dot product of the transpose of X_train_bias and X_train_bias
print("\nDeterminant of (X_train_bias^T.X_train_bias): ",det(X_train_bias.T.dot(X_train_bias)))


# Computes the dot product of the transpose of X_train_bias with itself
#  Denote the product as "z"

z = X_train_bias.T.dot(X_train_bias)

# Closed form (OLS) solution for weight vector w 
w = np.linalg.inv(z).dot(X_train_bias).T.dot(y)

print("\nThe weight vector:\n")
 
print(w)

print("\n----------------------------- Model Evaluation -----------------------------")



# Make prediction using the X_train_bias data matrix
# The predicted target vector should be named as "y_train_predicted"
y_train_predicted = X_train_bias.dot(w)

# Compute the MSE
print("Mean squared error:", mean_squared_error(y, y_predicted))


# Compute the r^2 score
print("Coefficient of determination r^2 variance score [1 is perfect prediction]:", r2_score(y, y_predicted))


('\nDeterminant of (X_train_bias^T.X_train_bias): ', 0.0)


LinAlgError: ignored

## Observation on the Performance of the Manually Coded OLS Solution

You might get the **Singularity matrix** error.

The determinant of the $X_{bias}^T.X_{bias}$ should be 0.

There must be colinearity in the columns of the data matrix X.

Find which columns are coliner.


## Applying OLS Method on Data Matrix With Colinearity in Columns

Solve the singularity problem can by adding small positive numbers on the diagonal of the $X_{bias}^T.X_{bias}$ matrix.

This regularization technique is known as **Ridge Regression**.


In [12]:
# Bayesian (Regularized) OLS Method for Linear Regression: Ridge Regression



# Add a bias term with the feature vectors to create a new data matrix "X_train_bias"
X_train_bias=np.c_[np.ones((X_train.shape[0],1)),X_train]


# Print the determinant of the dot product of the transpose of X_train_bias and X_train_bias
print("\nDeterminant of (X_train_bias^T.X_train_bias): ", det(X_train_bias.T.dot(X_train_bias)))


# Computes the dot product of the transpose of X_train_bias with itself
#  Denote the product as "z"

z = X_train_bias.T.dot(X_train_bias)


print("\n-------- Fixing the Singularity of (X_bias^T).X_bias ------------")

# Create a diagonal matrix that has the dimension of z; name the matrix as "diagonal"
f = np.full((15,),0.001)
diagonal = np.diagflat(f)

# Add small positive non-zero numbers on the diagonal
z = z + diagonal



# Closed form (OLS) solution for weight vector w 
w = np.linalg.inv(z).dot(X_train_bias.T).dot(y_train)



print("\nThe weight vector:\n")
print(w)


print("\n----------------------------- Model Evaluation -----------------------------")



# Make prediction using the X_train_bias data matrix
# The predicted target vector should be named as "y_train_predicted"
y_train_predicted = X_train_bias.dot(w)

# Compute the MSE
print("Mean squared error:", mean_squared_error(y_train, y_train_predicted))


# Compute the r^2 score
print("Coefficient of determination r^2 variance score [1 is perfect prediction]:", r2_score(y_train, y_train_predicted))


('\nDeterminant of (X_train_bias^T.X_train_bias): ', 0.0)

-------- Fixing the Singularity of (X_bias^T).X_bias ------------

The weight vector:

[[22.48521177]
 [-0.48574245]
 [ 0.70153482]
 [ 0.27672332]
 [ 0.70653526]
 [-1.99139527]
 [ 3.115727  ]
 [-0.17706139]
 [-3.04573205]
 [ 2.28270084]
 [-1.7925275 ]
 [-1.97994666]
 [ 1.12649546]
 [-3.62813639]
 [-0.48574245]]

----------------------------- Model Evaluation -----------------------------
('Mean squared error:', 21.6414127578023)
('Coefficient of determination r^2 variance score [1 is perfect prediction]:', 0.7508856358452931)


## Evaluate the Model Using Test Data - OLS Linear Regression

We evaluate the trained model on the test data.

Compute the MSE and $r^2$ score using the test data.

In [13]:
X_test=np.c_[np.ones((X_test.shape[0],1)),X_test]

# Make prediction using the X_train_bias data matrix
# The predicted target vector should be named as "y_train_predicted"
y_test_predicted = X_test.dot(w)

# Compute the MSE
print("Mean squared error:", mean_squared_error(y_test, y_test_predicted))


# Compute the r^2 score
print("Coefficient of determination r^2 variance score [1 is perfect prediction]:", r2_score(y_test, y_test_predicted))

('Mean squared error:', 24.291172758136184)
('Coefficient of determination r^2 variance score [1 is perfect prediction]:', 0.668758766951507)


# Part 2: Understanding the Singularity Issue and its Solution 

1) Why do you think the singularity matrix error occur while using OLS method on the “OLS_Data.csv” dataset?


Answer: The singularity error occured because the determinant was zero. We had colinearity in the dataset. The features X1 and X14 are having correlation of 1. Due to which the matrix was not full rank. 

2) To fix the singularity problem of the $X_{bias}^T.X_{bias}$ matrix what non-zero positive number did you add on its diagonal?


Answer: I added 0.001 to the diagonal of my matrix. 𝑋𝑇𝑏𝑖𝑎𝑠.𝑋𝑏𝑖𝑎𝑠 was denoted by 'z' in my code.

3) Add 100000 on the diagonal of the $X_{bias}^T.X_{bias}$ matrix and report the $MSE$ and the $r^2$ values for the training data set. Explain these results.


Answer: MSE for it 581.2107484879942 and  $r^2$  -5.745386629250396. We cannot have very high penalty/regularization because adding too high penality would be equal to adding noise in the data. Hence, we have very high MSE.

4) After adding 100000 on the diagonal of the $X_{bias}^T.X_{bias}$ matrix what change did you notice in the weights of the model?

Answer: The positive weights have decreased and the negative weights have increased.